# 7 합성곱 신경망 (CNN)

## 7.1 구조
합성공 계층 후에 폴링 계층 등장  
합성곱, 활성화 함수, 풀링 계층  
마지막에는 affine-softmax 조합 그대로 사용

## 7.2 합성곱 계층
완전연결 계층의 문제점은? '데이터의 형상이 무시된다'; 3차원 데이터->1차원 데이터
합성곱 계층은 형상 유지한다. 이미지도 3찬원 데이터, 다음 계층에도 3차원 데이터로 전달.  

입출력 데이터: 특징 맵
합성곱 계층의 입력 데이터: 입력 특징 맵
출력 데이터: 출력 특징 맵


### 7.2.1 합성곱 연산
입력 데이터 * 필터 --> 출력값
여기서의 필터는 완전연결 신경망에서의 '가중치'에 해당.

### 7.2.2 패딩
합성곱 연산을 수행하기 위하여 데이터 주변을 특정 값(예를 들면 0)으로 채우기도 한다. 이를 패딩이라 한다.
패딩 적용 이유? 출력 크기를 조정할 목적으로 사용한다. 
입력 데이터의 크기를 고정한 채로 그대로 출력 데이터의 크기

### 7.2.3 스트라이드
필터를 적용하는 위치의 간격
스트라이드를 키우면 출력 크기는 작아짐
패딩을 크게 하면 출력 크기가 커짐

입력 크기(H,W), 필터 크기를 (Fh,FW), 출력 크기를 (OH,OW), 패딩을 P, 스트라이드를 S라 하면 출력 크기는
OH = (H+2P-FH)/S + 1
OW = (W+2P-FW)/S + 1

### 7.2.4 3차원 데이터의 합성곱 연산

2차원 데이터에 비하여 길이 방향(채널 방향)으로 특징 맵이 늘어났다. 
*주의* 입력 데이터의 채널 수와 필터의 채널 수가 같아야 ㅎ나다.
필터 자체의 크기는 원하는 값으로 설정 간으하지만 모든 채널의 필터가 같은 크기여야 한다,

### 7.2.5 블록으로 생각
합성곱 연산의 출력으로 다수의 채널을 내보내기 위해서는 필터(가중치)사용
필터를 FN개 사용하면 출력 맵도  FN개 생성.
이 블록을 모아 다음 계층으로 넘기는 것이 CNN의 처리 흐름.

합성곱 연산에도 편향이 쓰인다. 현향은 채널 하나에 값 하나씩 구성된다.

## 7.3 풀링 계층
풀링: 세로 * 가로 방향의 공간을 줄이는 연산
최대 풀링,평균 풀링등이 있다. 이미지 인식 분야에서는 주로 최대 풀링을 사용한다.


### 7.3.1 
풀링 계층의 특징  
1. 학습해야 할 매개변수가 없다: 대상 영역에서 최댓값이나 평균을 취하는 명확한 처리이므로
2. 채널 수가 변하지 않는다
3. 입력의 변화에 영항을 받는다: 입력 데이터가 조금 변해도 결과는 잘 변하지 않음. 최대 등을 따르기 때문

## 7.4 합성곱/풀링 계층 구현하기
im2col 편의 함수 사용  (im2col: image to column)

In [1]:
# im2col 
import numpy as np

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


In [2]:
# 합성곱 계층
class Convolution:
    def __init__(self, W, b, stride = 1, pad=0):
        self.W = W
        self.b = b
        self.stride =stride
        self.pad = pad
        
    def forward(self,x):
        FN, C, FH, FW = self.W.shape
        N,C,H,W = x.shape
        out_h = int(1+(H+2*self.pad - FH)/self.stride) # 출력
        out_w = int(1+(W+2*self.pad - FW)/self.stride)
        
        
        col = im2col(x,FH,FW,self.stride,self.pad) #펼치기
        col_W = self.W.reshape(FN,-1).T #필터 펼치기
        out = np.dot(col,col_W)+self.b #행렬곱
        
        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2) #reshape
        
        return out
        
        
        

In [3]:
# 풀링 계층 구현
class Pooling:
    def __init__(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        N,C,H,W = x.shape
        out_h = int(1+(H-self.pool_h)/self.stride)
        out_w = int(1+(W-self.pool_w)/self.stride)
        
        #전개하기
        col = im2col(x, self.pool_h,self.pool_w, self.stride, self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)
        #최댓값
        out = np.max(col, axis=1)
        #모양 바꾸기
        out = out.reshape(N, out_h,out_w,C).transepose(0,3,1,2)
        
        return out        

입력 데이터를 im2col 사용하여 전개한 데이터와 필터를 세로로 1열로 전개하여 행렬 곱을 계산한다. 마지막으로 출력 데이터를 reshape 한다

풀링 계층 구현
1. 입력 데이터 전개
2. 행별로 최댓값을 구한다
3. 적절한 모양으로 성형

## 7.5 CNN 구현


In [4]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from deep_learning_from_scratch_master.layers import *
from deep_learning_from_scratch_master.gradient import numerical_gradient


class SimpleConvNet:
    
   
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
         #초기화 인수로 주어진 합성곱 계층의 하이퍼파라미터를 딕셔너리에서 꺼낸다.
         # 합성곱 계층의 출력 크기를 계산
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 매개변수 초기화
        """ 
        학습에 필요한 매개변수: 1번쨰 층 합성공 계층과 나머지 두 오나전연결 계층의 가중치와 편향. 
        이 매개변수들은 인스턴스 변수 params에 저장.
        1번째 층의 합성곱 계층 가중치 W1, 편향 b1
        2번째 층의 합성곱 계층 가중치 W2, 편향 b2        
                
        """
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # CNN을 구성하는 계층 생성한다
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

        # layers에 계층들을 차례로 추가.
        
        
    # 추론을 수행
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x


    
    # 손실 함수의 값을 구한다
    def loss(self, x, t):
        
        #x : 입력 데이터
        #t : 정답 레이블
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    
    
      #오차전역법으로 기울기를 구한다
    def gradient(self, x, t):
        #순전파와 역전파를 반복한다
      
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
    
   

## 7.6 CNN 시각화하기
입력으로 받은 이미지 데이터에서 '무엇을 보고 있는' 것인가

### 7.6.1 1번째 층
학습을 마친 필터는 규칙성이 있는 이미지가 된다  
에지나 블롭 등 원시적인 정보 추출 가능  
이런 원시적인 정보가 뒷단 계층에 전달

### 7.6.2 층 깊이에 따른 추출 정보 변화
계층이 깊어질수록 추출되는 정보(강하게 반응하는 뉴런)는 더 추상화 된다.
층이 깊어질수록 뉴런이 반응하는 대상이 단순한 모양에서 고급 정보로 변화해ㅎ간다; 사물의 의미를 이해하도록 변화한다.

## 7.7 대표적인 CNN
1. LeNet
2. AlexNet

## 7.8 정리
1. CNN 은 완전연결 계층 네트워크에 합성곱 계층과 풀링 계층을 새로 추가
2. 합성곱 계층과 풀링 계층은 im2col을 이용하면 간단하고 효율적으로 구현할 수 있다.
3. CNN 을 시각화해보면 계층이 깊어질수록 곡ㅂ 정보가 추출되는 모습 확인 가능
4. 딥러닝의 발전에는 빅데이터와 GPU가 크게 기여